In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('./data/act_train.csv')
df_people = pd.read_csv('./data/people.csv')
df_merge = pd.merge(df, df_people, how='inner', on='people_id')

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 50)

In [ ]:
df.tail(4)

In [ ]:
df['activity_category'].value_counts()

In [ ]:
df_act1 = df[df.activity_id.str.contains('^act1')].copy()
df_act2 = df[df.activity_id.str.contains('^act2')].copy()

In [ ]:
df_act1_type1 = df_act1[df_act1['activity_category']=='type 1'][['people_id', 'activity_id', 'date', 'activity_category', 'char_1', 
                                                                 'char_2', 'char_3', 'char_4', 'char_5', 'char_6', 'char_7', 'char_8', 
                                                                 'char_9', 'outcome']].copy()

In [ ]:
df_act2_type1 = df_act2[df_act2['activity_category']=='type 1'][['people_id', 'activity_id', 'date', 'activity_category', 'char_1', 
                                                                 'char_2', 'char_3', 'char_4', 'char_5', 'char_6', 'char_7', 'char_8', 
                                                                 'char_9', 'outcome']].copy()

In [ ]:
df_act1_type1.tail() #activity_category : type 1

In [ ]:
df_act2_type2 = df_act2[df_act2['activity_category']=='type 2'][['people_id', 'activity_id', 'date', 'activity_category', 'char_10', 'outcome']].copy()
df_act2_type3 = df_act2[df_act2['activity_category']=='type 3'][['people_id', 'activity_id', 'date', 'activity_category', 'char_10', 'outcome']].copy()
df_act2_type4 = df_act2[df_act2['activity_category']=='type 4'][['people_id', 'activity_id', 'date', 'activity_category', 'char_10', 'outcome']].copy()
df_act2_type5 = df_act2[df_act2['activity_category']=='type 5'][['people_id', 'activity_id', 'date', 'activity_category', 'char_10', 'outcome']].copy()
df_act2_type6 = df_act2[df_act2['activity_category']=='type 6'][['people_id', 'activity_id', 'date', 'activity_category', 'char_10', 'outcome']].copy()
df_act2_type7 = df_act2[df_act2['activity_category']=='type 7'][['people_id', 'activity_id', 'date', 'activity_category', 'char_10', 'outcome']].copy()

In [ ]:
df_act2_type1 # 없음

In [ ]:
df_act2_type2['char_10'].value_counts()

In [ ]:
df_act2_type2.head() #type 1    904683

In [ ]:
df_act2_type3['char_10'].value_counts() # 450

In [ ]:
df_act2_type3.tail()

In [ ]:
df_act2_type4['char_10'].value_counts() # 3315

In [ ]:
df_act2_type5['char_10'].value_counts() # 2747

In [ ]:
df_act2_type6['char_10'].value_counts() # type 110    4253

In [ ]:
df_act2_type6.tail()

In [ ]:
df_act2_type7['char_10'].value_counts() # type 194    3157

In [ ]:
import seaborn as sns

In [ ]:
import matplotlib.pylab as plt
plt.scatter(df_merge['char_38'],df_merge['outcome'],alpha=0.1)
plt.show()

In [ ]:
# df_test = df_merge[['people_id', 'outcome', 'char_38']].drop_duplicates() 쓰면 안된!!!

# feature corr

In [ ]:
a = df_merge.head().copy()

In [ ]:
a.replace('type ', '', inplace=True, regex=True)

In [ ]:
a.apply(pd.to_numeric, errors='ignore')

In [ ]:
a.dtypes.tail(40)

In [ ]:
a['char_8_y']

In [ ]:
a['char_8_y'].dtype

In [ ]:
pd.to_numeric()